# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откроем файл с данными и изучим общую информацию

In [1]:
import pandas as pd #импорт библиотеки pandas
# pymystem3 импортируется так:
from pymystem3 import Mystem
m = Mystem()
df = pd.read_csv('/datasets/data.csv') #чтение файла с данными и сохранение в df
display(df.head(10)) # получение первых 10 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
df.info()# получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Основной инструмент аналитика — `pandas`.


Прочитан файл `data.csv` из папки `/datasets` и сохранен в переменной `df`.

Изучена информация о таблице. В столбцах `days_employed` и `total_income` есть пропуски.

## Шаг 2. Предобработка данных

### Обработка пропусков

Для начала, чтобы получше познакомиться с данными, рассмотрим количество уникальных значений в каждом столбце. И найдем видимые ошибки.

In [3]:
df['children'].value_counts() #Уникальные значения столбца и их количество

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [4]:
df['dob_years'].value_counts() #Уникальные значения столбца и их количество

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [5]:
df['education'].value_counts() #Уникальные значения столбца и их количество

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [6]:
df['education_id'].value_counts() #Уникальные значения столбца и их количество

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [7]:
df['family_status'].value_counts() #Уникальные значения столбца и их количество

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [8]:
df['family_status_id'].value_counts() #Уникальные значения столбца и их количество

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [9]:
df['gender'].value_counts() #Уникальные значения столбца и их количество

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [10]:
df['income_type'].unique() #уникальные значения столбца

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [11]:
df['debt'].value_counts() #Уникальные значения столбца и их количество

0    19784
1     1741
Name: debt, dtype: int64

In [12]:
df['total_income'].value_counts() #Уникальные значения столбца и их количество

112874.418757    1
133912.272223    1
182036.676828    1
122421.963500    1
198271.837248    1
                ..
133299.194693    1
115080.782380    1
84896.781597     1
153838.839212    1
150014.128510    1
Name: total_income, Length: 19351, dtype: int64

In [13]:
df['purpose'].value_counts() #Уникальные значения столбца и их количество

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [14]:
df['days_employed'].value_counts() #Уникальные значения столбца и их количество

-986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413     1
Name: days_employed, Length: 19351, dtype: int64

В столбце `children` есть отрицательное число. Детей не может быть отрицательное количество. Заменим -1 на 0. Также у одного человека есть 20 детей, скорее всего это ошибка, и заменим 20 на 2.

In [15]:
df.loc[df['children'] == -1, 'children'] = 0 #замена -1 на 0
df.loc[df['children'] == 20, 'children'] = 2 #замена 20 на 2

Перепроверим данные:

In [16]:
df['children'].unique() #Уникальные значения столбца 

array([1, 0, 3, 2, 4, 5])

В столбце `education`, `family_status` разные регистры. Переведем все в нижний регистр и проверим результат.

In [17]:
df['education'] = df['education'].str.lower()
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [18]:
df['family_status'] = df['family_status'].str.lower()
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

В столбцах `days_employed` и `total_income` есть пропуски, как мы выяснили из первого пункта. Такое может происходить, если человек решил не заполнять данный пункт о себе или случайно пропустил, например. Также в столбце `days_employed`есть отрицательные значения. Такое может произойти из-за соединения нескольких таблиц.  Для начала переведем столбец `days_employed` к положительным числам с помощью метода `apply(abs)`.

In [19]:
df[df['days_employed'] > 0].sort_values(by = 'days_employed').head(10) #оценим минимальные значения в столбце

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20444,0,328728.720605,72,среднее,1,вдовец / вдова,2,F,пенсионер,0,96519.339647,покупка жилья для семьи
9328,2,328734.923996,41,высшее,0,женат / замужем,0,M,пенсионер,0,126997.497760,операции со своей недвижимостью
17782,0,328771.341387,56,среднее,1,женат / замужем,0,F,пенсионер,0,68648.047062,операции с коммерческой недвижимостью
14783,0,328795.726728,62,высшее,0,женат / замужем,0,F,пенсионер,0,79940.196752,на покупку своего автомобиля
7229,1,328827.345667,32,среднее,1,гражданский брак,1,F,пенсионер,0,122162.965695,сыграть свадьбу
5332,0,328834.274455,55,среднее,1,женат / замужем,0,F,пенсионер,0,49802.142892,профильное образование
5756,0,328842.364916,51,высшее,0,женат / замужем,0,F,пенсионер,0,189181.456572,недвижимость
13325,0,328862.571412,55,среднее,1,вдовец / вдова,2,F,пенсионер,0,99127.379599,на покупку автомобиля
3642,0,328874.270668,54,среднее,1,женат / замужем,0,F,пенсионер,0,202345.082322,строительство собственной недвижимости
3386,0,328895.862752,58,среднее,1,вдовец / вдова,2,F,пенсионер,0,130432.389679,образование


In [20]:
df[df['days_employed'] < 0].sort_values(by = 'days_employed').head(10) #оценим максимальные значения в столбце

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,-18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
4299,0,-17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
7329,0,-16593.472817,60,высшее,0,женат / замужем,0,F,сотрудник,0,124697.846781,заняться высшим образованием
17838,0,-16264.699501,59,среднее,1,женат / замужем,0,F,сотрудник,0,51238.967133,на покупку автомобиля
16825,0,-16119.687737,64,среднее,1,женат / замужем,0,F,сотрудник,0,91527.685995,покупка жилой недвижимости
3974,0,-15835.725775,64,среднее,1,гражданский брак,1,F,компаньон,0,96858.531436,сыграть свадьбу
1539,0,-15785.678893,59,высшее,0,не женат / не замужем,4,F,сотрудник,0,119563.851852,операции с коммерческой недвижимостью
4321,0,-15773.061335,61,среднее,1,гражданский брак,1,F,сотрудник,0,205868.585780,свадьба
7731,0,-15618.063786,64,среднее,1,женат / замужем,0,F,компаньон,0,296525.358574,высшее образование
15675,0,-15410.040779,65,высшее,0,женат / замужем,0,F,сотрудник,0,188800.068859,покупка жилой недвижимости


Заметим, что некоторые люди работают/работали 300000 дней, что говорит о том, что они работают/работали примерно 1000 лет, что невозможно. Человек в своей жизни может работать примерно до 30000 дней. Скорее всего некоторые данные могут быть представлены в часах. Соответственно переведем все значения столбца в положительные числа, а далее разделим числа, которые превышают 30000, на 24. 

In [21]:
df['days_employed'] = df['days_employed'].apply(abs) 
# data['days_employed'] >30000 позволяет выбрать строки с аномально большим стажем
df.loc[df['days_employed'] > 30000, 'days_employed'] = df.loc[df['days_employed'] > 30000, 'days_employed'] / 24 

Затем заполним пропуски в данных столбцах с группировкой по количеству детей. Среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства.Когда дело касается трудового стажа или заработка, разброс значений может быть большой, поэтому по отношению к упомянутым столбцам применим медиану.

In [22]:
for children in df['children'].unique(): #применим цикл перебора по количеству детей в таблице
    median = df.loc[df['children'] == children, 'days_employed'].median() #найдем медианное значение для определенного количества детей по трудовому стажу
    print(children,median) #вывод значений
    df.loc[(df['days_employed'].isna()) & (df['children'] == children), 'days_employed'] = median #замена пустых значений на медианное для определенного количества детей


1 1664.5049013073733
0 2626.036234562627
3 1765.0660443004722
2 1678.1320841476213
4 1905.879025041378
5 1231.5714864842407


In [23]:
#Аналогичные действия для ежемесячного дохода:
for children in df['children'].unique():
    median = df.loc[df['children'] == children, 'total_income'].median()
    print(children,median)
    df.loc[(df['total_income'].isna()) & (df['children'] == children), 'total_income'] = median

1 147943.0277792188
0 143937.20798209106
3 157221.54927459502
2 144600.7234995688
4 156135.2104343605
5 186351.41025073355


In [24]:
df.info()# получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В данном пункте выполнен анализ уникальных занчений по каждому столбцу. В столбце `children` заменены значений -1 и 20 на 0 и 2 соответсвенно. В столбце `education` все значения переведены в нижний регистр. В столбце `days_employed` применен модуль ко всем числам. В столбцах `days_employed`, `total_income` пропущенные значения заменены на медианные значения по группировке столбца количеству детей

### Замена типа данных

Сделаем столбец `days_employed` целочисленным методом `astype('int')`, так как в данном столбце находится общий трудовой стаж в днях, то есть целое число. 

In [25]:
df['days_employed']=df['days_employed'].astype('int') #перевод значений столбца в тип int
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**

В данном пункте бы выполнен перевод значений столбца `days_employed` в целочисленный тип. 

### Обработка дубликатов

В пункте 2.1 было посчитано количество уникальных значений для каждого столбца, все слова переведены в нижний регистр для удобства поиска явных дубликатов, посчитаем их количество последовательным применением методов `duplicated()`,`sum()`.

In [26]:
df.duplicated().sum() #посчитаем количество дубликатов

71

Удалим дубликаты методом `drop_duplicates()`. При вызове метода `drop_duplicates()` вместе с повторяющимися строками удаляются их индексы, поэтому используется с методом `reset_index()`.

In [27]:
df = df.drop_duplicates().reset_index(drop = True)

In [28]:
df.duplicated().sum() #проверим результат

0

**Вывод**

В данном пункте были удалены полные совпадения явных дубликатов. Дубликаты могли появится из-за разных регистров, некорректного количества детей, пропусков.

### Лемматизация

Проанализируем столбец purpose, так как можно заметить, что причины для получения кредита меньше,чем количество уникальных значений столбца, так как одни и те же причины записаны по-разному. 

Одна из библиотек с функцией лемматизации на русском языке — pymystem3, разработана сотрудниками Яндекса. 

In [30]:
# функция для лемматизации столбца purpose
def categorize(purpose):
    purpose_lemmas = m.lemmatize(purpose) 
    if 'недвижимость' in purpose_lemmas or 'жилье' in purpose_lemmas:
        return 'недвижимость'
    elif 'образование' in purpose_lemmas:
        return 'образование'
    elif 'автомобиль' in purpose_lemmas:
        return 'автомобиль'
    elif 'свадьба' in purpose_lemmas:
        return 'свадьба'
    else:
        return 'прочее'

In [31]:
df['general_purpose'] = df['purpose'].apply(categorize) #добавляем новый столбец в таблицу  
df['general_purpose'].value_counts() #выводим количество уникальных значений

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: general_purpose, dtype: int64

**Вывод**

В данном пункте создан отдельный столбец с леммами столбца `purpose`.

### Категоризация данных

Думаю, одна из важных категорий — тип занятости человека, чтобы нацелиться на удержание данных людей в категории. 

In [32]:
df_category = df[['income_type']].drop_duplicates()
df_category.head()

,income_type
0,сотрудник
4,пенсионер
5,компаньон
26,госслужащий
3132,безработный


Также следует понимать в каком семейном положении находится человек. 

In [33]:
df[['family_status']].drop_duplicates()

,family_status
0,женат / замужем
4,гражданский брак
18,вдовец / вдова
19,в разводе
24,не женат / не замужем


При целях привлечения новых клиентов и удержании имеющихся необходимо понимать для каких целей люди берут кредит. 

In [34]:
df[['general_purpose']].drop_duplicates()

,general_purpose
0,недвижимость
1,автомобиль
3,образование
4,свадьба


**Вывод**

В данном пункте подобраны категории, на которые следует обратить внимание при привлечении новых клиентов и удержании имеющихся.

## Шаг 3. Ответим на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [35]:
def children_count(row):
    children = row['children']
    debt = row['debt']
    
    if children == 0:
        if debt == 0:
            return 'нет детей'
    if children == 1:
        if debt == 0:
            return '1 ребенок'
    if children == 2:
        if debt == 0:
            return '2 ребенка'
    if children == 3:
        if debt == 0:
            return '3 ребенка'
    if children == 4:
        if debt == 0:
            return '4 ребенка'
    if children == 5:
        if debt == 0:
            return '5 детей'
    return 'не отдают вовремя кредит'
row_values =[1, 0]
row_columns =['children', 'debt']
row=pd.Series(data=row_values, index=row_columns)


df['children_debt']=df.apply(children_count, axis = 1)
print(df['children_debt'].value_counts())

нет детей                   13074
1 ребенок                    4364
2 ребенка                    1926
не отдают вовремя кредит     1741
3 ребенка                     303
4 ребенка                      37
5 детей                         9
Name: children_debt, dtype: int64


**Вывод**

Как можно заметить люди, у которых 5 детей редко вовремя отдают кредит. Малый процент людей,отдающих вовремя кредит, составили те, кто имеют 3-4 ребенка. Чаще отдают вовремя кредит люди без детей. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [36]:
def family_count(row):
    family_status = row['family_status']
    debt = row['debt']
    
    if family_status == 'женат / замужем':
        if debt == 0:
            return 'женат / замужем'
    if family_status == 'гражданский брак':
        if debt == 0:
            return 'гражданский брак'
    if family_status == 'вдовец / вдова':
        if debt == 0:
            return 'вдовец / вдова'
    if family_status == 'в разводе':
        if debt == 0:
            return 'в разводе'
    if family_status == 'Не женат / не замужем':
        if debt == 0:
            return 'не женат / не замужем'
    
    return 'не отдают вовремя кредит'
row_values =['Не женат / не замужем', 0]
row_columns =['family_status', 'debt']
row =pd.Series(data = row_values, index = row_columns)


df['family_debt'] = df.apply(family_count, axis = 1)
print(df['family_debt'].value_counts())

женат / замужем             11408
не отдают вовремя кредит     4277
гражданский брак             3763
в разводе                    1110
вдовец / вдова                896
Name: family_debt, dtype: int64


**Вывод**

Люди,находящиеся в браке чаще отдают вовремя кредит, чем одинокие.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [37]:
def total_count(row):
    total_income = row['total_income']
    debt = row['debt']
    
    if total_income < 40000:
        if debt == 0:
            return 'низкий доход'
    if 40000 <= total_income <=80000:
        if debt==0:
            return 'средний доход'
    if total_income > 80000:
        if debt == 0:
            return 'высокий доход'
    
    return 'не отдают вовремя кредит'
row_values =[20000,0]
row_columns =['total_income', 'debt']
row =pd.Series(data = row_values, index = row_columns)


df['total_debt'] = df.apply(total_count, axis = 1)
print(df['total_debt'].value_counts())

высокий доход               17611
средний доход                1987
не отдают вовремя кредит     1741
низкий доход                  115
Name: total_debt, dtype: int64


**Вывод**

Очевидно, что люди с большим доходом чаще отдают кредит вовремя.

- Как разные цели кредита влияют на его возврат в срок?

In [38]:
def purpose_count(row):
    general_purpose = row['general_purpose']
    debt = row['debt']
    
    if general_purpose == 'недвижимость':
        if debt == 0:
            return 'недвижимость'
    if general_purpose == 'автомобиль':
        if debt == 0:
            return 'автомобиль'
    if general_purpose == 'свадьба':
        if debt == 0:
            return 'свадьба'
    if general_purpose == 'обучение':
        if debt == 0:
            return 'обучение'
    
    
    return 'не отдают вовремя кредит'
row_values =['обучение',0]
row_columns =['general_purpose', 'debt']
row =pd.Series(data = row_values, index = row_columns)


df['purpose_debt'] = df.apply(purpose_count, axis = 1)
print(df['purpose_debt'].value_counts())

недвижимость                10029
не отдают вовремя кредит     5384
автомобиль                   3903
свадьба                      2138
Name: purpose_debt, dtype: int64


**Вывод**

Чаще люди отдают вовремя кредит за недвижимоть, реже всего за свадьбу. За обучение не отдают вовремя кредит.

## Шаг 4. Общий вывод

В ходе работы была изучена таблица о заемщиках и информация о данной таблицы. Просмотрены уникальные значения в каждом столбце, найдены и исправлены некорректные значения, все буквы переведены в нижний регистр, чтобы удобнее избавиться от дубликатов. Также были заполнены пропущенные значения. Значения столбца количнества отработанных дней были переведены в целочисленный формат. Также был создан отдельный столбец с конкретными причинами кредита. Выявлены категории по занятости людей,причинам кредита и семейному положени. Найдены зависимости возврата кредита в срок. люди, у которых 5 детей редко вовремя отдают кредит. Малый процент людей,отдающих вовремя кредит, составили те, кто имеют 3-4 ребенка. Чаще отдают вовремя кредит люди без детей. Люди,находящиеся в браке чаще отдают вовремя кредит, чем одинокие. Очевидно, что люди с большим доходом чаще отдают кредит вовремя. Чаще люди отдают вовремя кредит за недвижимоть, реже всего за свадьбу. За обучение не отдают вовремя кредит.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.